In [512]:
import json
import sys
sys.path.append('../..')
import pandas as pd
import ta
from utilities.data_manager import ExchangeDataManager
pd.options.mode.chained_assignment = None  # default='warn'


In [603]:
class Strategy():
    def __init__(
        self,
        pair,
        type=["long"],
        params={},
    ):
        self.df_pair = None
        self.df = None
        self.pair = pair
        self.initial_wallet = 1000
        self.use_long = "long" in type
        self.use_short = "short" in type
        self.params = params
        self.result_df = None

    def get_pair_data(self, timeframe, start = 2050, end = 2050):
        exchange = ExchangeDataManager(
            exchange_name=exchange_name,
            path_download="./database/exchanges"
        )

        self.df_pair = exchange.load_data(self.pair, timeframe, start, end)

    def populate_indicators2(self):
        params = self.params
        df = self.df_pair.copy()
        df.drop(
            columns=df.columns.difference(['open','high','low','close','volume']),
            inplace=True
        )
        df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
        df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
        df['mrat'] = df['fast_ma'] / df['slow_ma']
        df['mean_mrat'] = ta.trend.sma_indicator(close=df['mrat'], window=params["mean_mrat_lenght"])
        df['stdev_mrat'] = df['mrat'].rolling(params["mean_mrat_lenght"]).std(ddof=0)
        df['open_long_signal'] = df['mean_mrat'].shift(1) - df['mrat'].shift(1) >= params['sigma_open'] * df['stdev_mrat'].shift(1)
        df['close_long_signal'] = df['mrat'].shift(1) - df['mean_mrat'].shift(1) >= params['sigma_close'] * df['stdev_mrat'].shift(1)

        df_signal = df.loc[
            df["open_long_signal"] | df["close_long_signal"],
            ["open_long_signal", "close_long_signal","open", "close"]
        ]
        df_signal["open_signal_lag"] = df_signal["open_long_signal"].shift(fill_value=False)
        df_signal["close_signal_lag"] = df_signal["close_long_signal"].shift(fill_value=False)
        df_first_signal  = df_signal[
            (~ df_signal["open_signal_lag"] & (df_signal["open_long_signal"] | df_signal["open_long_signal"].isnull())) |
            (~ df_signal["close_signal_lag"] & df_signal["close_long_signal"])
         ]
        df_first_signal["open_signal_lag"] = df_first_signal["open_long_signal"].shift(fill_value=False)
        df_first_signal["close_signal_lag"] = df_first_signal["close_long_signal"].shift(fill_value=False)

        df_order_tmp = df_first_signal[
            (df_first_signal["open_long_signal"] & (~df_first_signal["open_signal_lag"] | df_first_signal["open_long_signal"].isnull())) |
            (df_first_signal["close_long_signal"] & ~ df_first_signal["close_signal_lag"])
        ]
        df_order = df_order_tmp.loc[
            ~ ( ~ df_order_tmp["close_signal_lag"] & ~ df_order_tmp["open_signal_lag"] & df_order_tmp["close_long_signal"])
        ]
        df_order["order_number"] = df_order["open_long_signal"].cumsum()
        df_order["open_lag"] = df_order["open"].shift(-1)
        df_order["open_order"] = df_order["open"].shift()
        df_order.loc[df_order["open_long_signal"], "open_order"] = df_order.loc[df_order["open_long_signal"], "open"]
        df_pair = df[["open", "close", "low", "high", "mrat", "mean_mrat", "stdev_mrat", "open_long_signal", "close_long_signal"]]

        leverage = params["leverage"]  # Fixed leverage
        maintenance_margin_percent = 0.004
        wallet = 1000  # Initial wallet balance
        quantity = 0  # Initial quantity

        # Ensure the DataFrame has 'quantity' and 'trade_result' columns initialized
        df_order['quantity'] = 0.0
        df_order['trade_result'] = 0.0
        df_order['trade_result_pct'] = 0.0

        # Iterating over DataFrame rows to process trading signals
        for i, row in df_order.iterrows():
            # Check if there is a signal to open a long position
            if row['open_long_signal']:
                # Calculate the new quantity based on the current wallet and leverage
                quantity = wallet * leverage / row['open']
                # Update the 'quantity' column with the new quantity
                df_order.at[i, 'quantity'] = quantity
                # No change in wallet yet as the position has just opened
                df_order.at[i, 'wallet'] = wallet
                # Track the price at which the position was opened
                open = row['open']
            elif row['close_long_signal']:
                # Calculate the trade result based on the difference between current and open price
                trade_result = (row['open'] - open) * quantity
                # Update the 'trade_result' column with the result of the closed trade
                df_order.at[i, 'trade_result'] = trade_result
                df_order.at[i, 'trade_result_pct'] = trade_result / wallet * 100
                # Update the wallet with the result of the trade
                wallet += trade_result
                # Reset quantity as the trade is closed
                df_order.at[i, 'quantity'] = quantity
                quantity = 0

            # Update the wallet and quantity for the current row
            df_order.at[i, 'wallet'] = wallet

        df_order_tmp = df_order[
            ["order_number", "quantity", "trade_result", "trade_result_pct", "wallet", "open_order"]
        ]
        df_order_final = df_pair.join(df_order_tmp)

        f = df_order_final['order_number'].ffill()
        b = df_order_final['order_number'].bfill()

        df_order_final['order_number'] = df_order_final['order_number'].mask(f == b, f)

        f = df_order_final['open_order'].ffill()
        b = df_order_final['open_order'].bfill()

        df_order_final['open_order'] = df_order_final['open_order'].mask(f == b, f)
        df_order_final['wallet'] = df_order_final['wallet'].ffill()
        #df_order_final["hypothetical_wallet"] = df_order_final["wallet"] + df_order_final["quantity"] * (df_order_final['open'] - df_order_final["open_order"])
        df_order_final["hypothetical_wallet"] = df_order_final["wallet"].shift() + df_order_final["quantity"] * (df_order_final['open'] - df_order_final["open_order"])
        df_order_final["hypothetical_low_result"] =  ((df_order_final["quantity"] * df_order_final["low"]) - df_order_final["wallet"]) / df_order_final["wallet"]
        df_order_final["drawdown"] = (df_order_final["low"] - df_order_final["open_order"]) / df_order_final["open_order"] * 100 * leverage
        df_order_final["is_liquidated"] = df_order_final['hypothetical_wallet'] < (df_order_final["wallet"] / leverage) * maintenance_margin_percent

        self.df = df_order_final


    def populate_indicators(self):
        params = self.params
        df = self.df_pair.copy()
        df.drop(
            columns=df.columns.difference(['open','high','low','close','volume']),
            inplace=True
        )

        # -- Populate indicators --
        df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
        df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
        df['mrat'] = df['fast_ma'] / df['slow_ma']
        df['mean_mrat'] = ta.trend.sma_indicator(close=df['mrat'], window=params["mean_mrat_lenght"])
        df['stdev_mrat'] = df['mrat'].rolling(params["mean_mrat_lenght"]).std(ddof=0)
        df['open_long_signal'] = df['mean_mrat'].shift(1) - df['mrat'].shift(1) >= params['sigma_open'] * df['stdev_mrat'].shift(1)
        df['close_long_signal'] = df['mrat'].shift(1) - df['mean_mrat'].shift(1) >= params['sigma_close'] * df['stdev_mrat'].shift(1)

        df["is_liquidated"] = False
        df["order_open"] = False
        # Trading logic
        order_open = False
        current_order_number = 0
        open_price = 0
        quantity = 0
        trade_result = 0
        # Constants and Initialization
        initial_wallet = self.initial_wallet
        leverage = params["leverage"]  # Fixed leverage
        maintenance_margin_percent = 0.004
        wallet = initial_wallet
        max_equity = initial_wallet  # To track the max equity before a new trade
        max_drawdown = 0

        for i in df.index:
            if df.loc[i, 'open_long_signal'] and not order_open:
                # Open a new order
                current_order_number += 1
                order_open = True
                open_price = df.loc[i, 'open']
                open_wallet = df.loc[i, 'wallet']
                quantity = (wallet / open_price) * leverage
                df.loc[i, 'order_number'] = current_order_number
                df.loc[i, 'order_open'] = order_open

            # Assign order_number to all rows of the current order
            if order_open:
                df.loc[i, 'order_number'] = current_order_number
                df.loc[i, 'order_open'] = order_open
                # Calculate hypothetical_wallet
                hypothetical_wallet = wallet + quantity * (df.loc[i, 'open'] - open_price)
                df.loc[i, 'hypothetical_wallet'] = hypothetical_wallet
                df.loc[i, 'quantity'] = quantity

                # Check for liquidation
                maintenance_margin = (wallet / leverage) * maintenance_margin_percent
                if hypothetical_wallet < maintenance_margin:
                    df.loc[i, 'is_liquidated'] = True
                    df.loc[i, 'trade_result'] = hypothetical_wallet - wallet
                    wallet = 0  # Update wallet with the loss
                    order_open = False  # Close the order

            # Close the order
            if df.loc[i, 'close_long_signal'] and order_open:
                trade_result = quantity * (df.loc[i, 'open'] - open_price)
                trade_result_perc = trade_result / wallet * 100
                wallet += trade_result  # Update wallet with the profit or loss
                order_open = False  # Close the order
                df.loc[i, 'trade_result'] = trade_result
                df.loc[i, 'trade_result_perc'] = trade_result_perc

            # Set wallet to current wallet value
            df.loc[i, 'wallet'] = wallet

        df["drawdown"] = (df["hypothetical_wallet"] - df["wallet"]) / df["wallet"] * 100

        self.df = df


    def get_result_df(self):
        try:
            df = self.df
            if df is not None:
                total_trades = df.order_number.max()
                final_wallet_amount = df.loc[df["open_long_signal"], "wallet"].tail(1)
                total_profit = final_wallet_amount - self.initial_wallet
                total_profit_perc = total_profit / self.initial_wallet * 100
                avg_trade_profit_perc = df["trade_result_pct"].dropna().mean()
                avg_trade_profit = df["trade_result"].dropna().mean()
                max_drawdown = df["drawdown"].min()

                result_df = pd.DataFrame(
                    {
                        "params": str(self.params),
                        "final_wallet_amount": final_wallet_amount,
                        "total_profit": total_profit,
                        "total_profit_perc": total_profit_perc,
                        "total_trades": total_trades,
                        "avg_trade_profit_perc": avg_trade_profit_perc,
                        "avg_trade_profit": avg_trade_profit,
                        "max_drawdown": max_drawdown,
                    }
                )

                return result_df
            else:
                return None
        except Exception as e:
            print(e)
            print(self.params)
            return None

In [517]:
params = {
    "fast_ma": 60,
    "slow_ma": 60,
    "sigma_open": 1,
    "sigma_close": 2.9,
    "mean_mrat_lenght": 60,
    "leverage": 1
}

pair = "API3/USDT:USDT"
exchange_name = "binance"
tf = '15m'
oldest_pair = "API3/USDT:USDT"
start_date = "2023-01-01 00:00:00"
end_date = "2023-12-31 00:00:00"

In [515]:
strat = Strategy(pair=pair, params=params)
strat.get_pair_data(timeframe=tf, start=start_date)
df1 = strat.populate_indicators()

In [ ]:
strat = Strategy(pair=pair, params=params)
strat.get_pair_data(timeframe=tf, start=start_date, end=end_date)
df2 = strat.populate_indicators2()

# Check best params result

## API3

# TradingView consistence check

In [809]:
pair = "API3/USDT:USDT"
exchange_name = "binance"
tf = '15m'

In [810]:
df_result = pd.read_csv("results/API3_binance_15m_2023-01-01_2023-12-31_2.csv")

In [811]:
df_result.sort_values("final_wallet_amount", ascending=False).head(20)

,date,params,final_wallet_amount,total_profit,total_profit_perc,total_trades,avg_trade_profit_perc,avg_trade_profit,max_drawdown
134417,2023-12-27 14:15:00,"{'fast_ma': 45, 'slow_ma': 130, 'sigma_open': ...",6110.124098,5110.124098,511.012410,32.0,3.322065,81.113081,-33.977456
126494,2023-12-27 13:30:00,"{'fast_ma': 40, 'slow_ma': 130, 'sigma_open': ...",5721.771949,4721.771949,472.177195,41.0,2.380483,58.293481,-19.609756
126514,2023-12-27 13:15:00,"{'fast_ma': 40, 'slow_ma': 130, 'sigma_open': ...",5672.590534,4672.590534,467.259053,39.0,2.468339,60.682994,-19.844358
917,2023-12-28 16:00:00,"{'fast_ma': 5, 'slow_ma': 60, 'sigma_open': 2....",5597.140374,4597.140374,459.714037,80.0,1.174467,28.811047,-22.495274
134416,2023-12-27 14:15:00,"{'fast_ma': 45, 'slow_ma': 130, 'sigma_open': ...",5531.853541,4531.853541,453.185354,34.0,2.866569,67.639605,-29.310345
126534,2023-12-27 12:45:00,"{'fast_ma': 40, 'slow_ma': 130, 'sigma_open': ...",5509.907257,4509.907257,450.990726,38.0,2.492006,60.132097,-19.844358
126517,2023-12-27 13:15:00,"{'fast_ma': 40, 'slow_ma': 130, 'sigma_open': ...",5330.416552,4330.416552,433.041655,34.0,2.793497,64.633083,-28.819444
134068,2023-12-27 14:15:00,"{'fast_ma': 45, 'slow_ma': 125, 'sigma_open': ...",5322.116960,4322.116960,432.211696,28.0,3.435533,78.583945,-28.819444
104429,2023-12-24 00:15:00,"{'fast_ma': 30, 'slow_ma': 55, 'sigma_open': 2...",5302.126854,4302.126854,430.212685,36.0,2.818852,59.535030,-18.403305
134415,2023-12-27 14:15:00,"{'fast_ma': 45, 'slow_ma': 130, 'sigma_open': ...",5277.026209,4277.026209,427.702621,37.0,2.555983,58.589400,-29.310345


In [762]:
params = {
    "fast_ma": 5,
    "slow_ma": 60,
    "sigma_open": 2.5,
    "sigma_close": 2.7,
    "mean_mrat_lenght": 60,
    "leverage": 1
}

pair = "API3/USDT:USDT"
exchange_name = "binance"
tf = '15m'
start_date = "2023-12-01 00:00:00"
end_date = "2024-03-24 01:00:00"

strat = Strategy(pair=pair, params=params)
strat.get_pair_data(timeframe=tf, start=start_date, end=end_date)
strat.populate_indicators2()
df_result_tmp = strat.get_result_df()
df = strat.df

In [763]:
df.loc[df["order_number"] == 1].iloc[[0,-1],:]

,open,close,low,high,mrat,mean_mrat,stdev_mrat,open_long_signal,close_long_signal,order_number,quantity,trade_result,trade_result_pct,wallet,open_order,hypothetical_wallet,hypothetical_low_result,drawdown,is_liquidated
date,,,,,,,,,,,,,,,,,,,
2023-12-02 17:15:00,1.5588,1.5575,1.5568,1.5605,1.003348,1.023243,0.007682,True,False,1.0,641.519117,0.00000,0.000000,1000.00000,1.5588,NaN,-0.001283,-0.128304,False
2023-12-05 00:30:00,1.5750,1.5856,1.5664,1.5908,1.037881,0.978960,0.018748,False,True,1.0,641.519117,10.39261,1.039261,1010.39261,1.5588,1010.39261,-0.005460,0.487555,False


In [764]:
df.loc[df["order_number"] == df["order_number"].max() - 1].iloc[[0,-1],:]

,open,close,low,high,mrat,mean_mrat,stdev_mrat,open_long_signal,close_long_signal,order_number,quantity,trade_result,trade_result_pct,wallet,open_order,hypothetical_wallet,hypothetical_low_result,drawdown,is_liquidated
date,,,,,,,,,,,,,,,,,,,
2024-03-12 06:30:00,3.8007,3.7861,3.7805,3.8172,0.983506,1.007186,0.006351,True,False,24.0,534.837577,0.000000,0.000000,2032.757180,3.8007,2032.757180,-0.005315,-0.531481,False
2024-03-17 03:30:00,3.2426,3.2630,3.2417,3.2768,1.003626,0.961099,0.015625,False,True,24.0,534.837577,-298.492852,-14.684137,1734.264328,3.8007,1734.264328,-0.000278,-14.707817,False


In [765]:
df_result_tmp

,params,final_wallet_amount,total_profit,total_profit_perc,total_trades,avg_trade_profit_perc,avg_trade_profit,max_drawdown
date,,,,,,,,
2024-03-18 02:00:00,"{'fast_ma': 5, 'slow_ma': 60, 'sigma_open': 2....",1734.264328,734.264328,73.426433,25.0,1.220592,14.984986,-28.693689


✅ Same first order as TV
✅ Same trade result
✅ Same profit

## Best params 2024 performance

In [812]:
top_params = df_result.sort_values("final_wallet_amount", ascending=False).iloc[:100]["params"]

In [813]:
train_start_date = "2023-01-01 00:00:00"
train_end_date = "2023-12-31 00:00:00"
test_start_date = "2024-01-01 00:00:00"
test_end_date = "2024-03-22 00:00:00"

result = []
for param in top_params:
    param = param.replace("'", "\"")
    param = json.loads(param)
    param["leverage"] = 1
    strat = Strategy(pair=pair, params=param)
    strat.get_pair_data(timeframe=tf, start=test_start_date, end=test_end_date)
    strat.populate_indicators2()
    df_result_tmp = strat.get_result_df()
    result.append(df_result_tmp)

In [814]:
result_dfs = pd.concat(result)

In [815]:
result_dfs.sort_values("final_wallet_amount", ascending=False)

,params,final_wallet_amount,total_profit,total_profit_perc,total_trades,avg_trade_profit_perc,avg_trade_profit,max_drawdown
date,,,,,,,,
2024-03-15 13:00:00,"{'fast_ma': 40, 'slow_ma': 125, 'sigma_open': ...",2236.247990,1236.247990,123.624799,8.0,6.451376,82.416533,-13.752607
2024-03-15 13:45:00,"{'fast_ma': 45, 'slow_ma': 125, 'sigma_open': ...",2110.096710,1110.096710,111.009671,7.0,6.422197,85.392055,-22.038161
2024-03-15 13:30:00,"{'fast_ma': 45, 'slow_ma': 125, 'sigma_open': ...",2108.017696,1108.017696,110.801770,7.0,6.431124,85.232130,-21.858732
2024-03-15 13:45:00,"{'fast_ma': 45, 'slow_ma': 125, 'sigma_open': ...",2025.129080,1025.129080,102.512908,7.0,6.031854,78.856083,-22.038161
2024-03-15 13:30:00,"{'fast_ma': 45, 'slow_ma': 125, 'sigma_open': ...",2023.133782,1023.133782,102.313378,7.0,6.040575,78.702599,-21.858732
...,...,...,...,...,...,...,...,...
2024-03-19 08:00:00,"{'fast_ma': 30, 'slow_ma': 50, 'sigma_open': 2...",682.338462,-317.661538,-31.766154,11.0,-1.729583,-15.126740,-23.187643
2024-03-19 08:30:00,"{'fast_ma': 30, 'slow_ma': 50, 'sigma_open': 2...",682.014789,-317.985211,-31.798521,14.0,-1.316925,-11.777230,-23.187643
2024-03-19 08:30:00,"{'fast_ma': 30, 'slow_ma': 50, 'sigma_open': 2...",653.619763,-346.380237,-34.638024,11.0,-1.793757,-16.494297,-24.030101


In [782]:
result_dfs.sort_index().iloc[0]["params"]

"{'fast_ma': 35, 'slow_ma': 50, 'sigma_open': 2.8, 'sigma_close': 2.9, 'mean_mrat_lenght': 50, 'leverage': 1}"